# Install Dependencies

In [ ]:
import pandas as pd
from sklearn.metrics import pair_confusion_matrix
import numpy as np

# Data Exploration

### Checking if Cho is a proper subset of KEGG

In [ ]:



cho_data = pd.read_csv('../dataset/Cho_Ours.txt', sep='\t')
cho_genes = cho_data.iloc[:, 0]

kegg_data = pd.read_csv('../dataset/Kegg_Subcluster.txt', sep='\t')
kegg_genes = kegg_data.iloc[:, 0]

cho_genes_in_kegg = cho_genes.isin(kegg_genes)
cho_genes_not_in_kegg = cho_genes[~cho_genes_in_kegg]
missing_genes_count = cho_genes_not_in_kegg.count()
print("Are all cho_genes present in kegg_genes?", cho_genes_in_kegg.all())
print("Number of missing genes in kegg_genes:", missing_genes_count)
print(cho_genes_not_in_kegg)

Are all cho_genes present in kegg_genes? False
Number of missing genes in kegg_genes: 21
18     YLR273c
34     YKL042w
39     YNL225c
43     YLR313c
44     YGR041w
61     YJL173c
80     YMR179w
85     YDL227c
86     YNL173c
101    YMR198w
103    YDR356w
111    YDL197c
123    YIR017c
130    YKL049c
131    YCL014w
132    YOR188w
140    YLR014c
150    YBR138c
154    YJR092w
155    YLR353w
168    YDL048c
Name: Gene, dtype: object


# Formulation of the Modified Rand Index

In [ ]:
import itertools

def modified_rand_score(labels_true, labels_pred, nodes_true_list, nodes_pred_list):
    """
    Compute the Modified Rand Index (MRI), an extension of the Rand Index
    that accounts for missing elements between partitions.

    Parameters
    ----------
    labels_true : list
        True labels corresponding to the nodes.
    labels_pred : list
        Predicted labels corresponding to the nodes.
    nodes_true_list : list
        An ordered list of nodes corresponding to labels_true.
    nodes_pred_list : list
        An ordered list of nodes corresponding to labels_pred.

    Returns
    -------
    mri : float
        The Modified Rand Index score.
    """

    n = len(labels_true)
    total_pairs = n * (n - 1) // 2  # All possible unique node pairs
    n_00, n_11, n_xx = 0, 0, 0  # Initialize counts

    # Convert lists to sets for fast lookup
    nodes_true_set = set(nodes_true_list)
    nodes_pred_set = set(nodes_pred_list)

    # Identify common nodes between the two partitions
    common_nodes = nodes_true_set.intersection(nodes_pred_set)

    # Iterate over all node pairs
    for i, j in itertools.combinations(range(n), 2):  # Iterate over all unique pairs
        node_i, node_j = nodes_true_list[i], nodes_true_list[j]

        # If either node is missing from the common set, count this pair as n_xx.
        if node_i not in common_nodes or node_j not in common_nodes:
            n_xx += 1
        else:
            same_true = (labels_true[i] == labels_true[j])
            same_pred = (labels_pred[i] == labels_pred[j])
            if same_true and same_pred:
                n_11 += 1  # Agreeing pair (same in both)
            elif not same_true and not same_pred:
                n_00 += 1  # Agreeing pair (different in both)

    # Compute MRI Score
    if total_pairs == 0:
        return 1.0  # Perfect match in trivial cases

    MRI = (n_00 + n_11 + n_xx) / total_pairs
    return MRI